In [ ]:
import gensim
import gensim.models.keyedvectors as word2vec
import numpy as np
model = gensim.models.KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin", binary=True)

def cosine_sim(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def avg_sentence(sentence, wv):
    v = np.zeros(300)
    for w in sentence:
        if w in wv:
            v += wv[w]
    return v / len(sentence)

In [ ]:
from sklearn.metrics import accuracy_score
import nltk
from nltk.corpus import stopwords
stop_words = stopwords.words('english')


In [ ]:
import csv
with open('zero_lbl.txt') as tsvfile, open('zero_label.txt', 'w') as t:
    reader = csv.reader(tsvfile, delimiter='\t')
    temp = csv.writer(t, delimiter="\t")
    count = 0
    lbl_count = {"0":0, "1":0}
    y_true, y_predict = [], []
    for row in reader:
        #if count < 0:
        #    break    
        #print(row[0], "3: ", row[3],"4: ", row[4],"5: ", row[5])
        sen1 = [word for word in ("".join(x for x in row[3] if x.isalnum() or x == " ")).split() if word not in stop_words]
        sen2 = [word for word in ("".join(x for x in row[4] if x.isalnum() or x == " ")).split() if word not in stop_words]
        #sen2 = "".join(x for x in row[4] if x.isalnum() or x == " ")
        vec1 = avg_sentence(sen1, model.wv)
        vec2 = avg_sentence(sen2, model.wv)
        sim = cosine_sim(vec1, vec2)
        y_predict.append(1 if sim > 0.9 else 0)
        y_true.append(int(row[5]))
        row[5] = "1" if sim > 0.9 else "0"
        temp.writerow(row)
        print(row[3], row[4], sim)
        lbl_count[row[5]] += 1
        count += 1    
    print(accuracy_score(y_true, y_predict))
    print(lbl_count, count)